## Motion Correction using the NoRMCorre algorithm

Can use both rigid and piecewise motion correction. The original motion correction algorithm is described in this paper:
https://www.sciencedirect.com/science/article/pii/S0165027017302753?via%3Dihub

More details and tips here: https://caiman.readthedocs.io/en/master/CaImAn_Tips.html#motion-correction-tips

And an example pipeline here:  https://github.com/flatironinstitute/CaImAn/blob/6c5c3b6117b71b6e8b44f62fc26fd3b3d914de12/demos/notebooks/demo_motion_correction.ipynb

In [17]:
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import inspect_correlation_pnr

import logging
from fancylog import fancylog
import fancylog as package


try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl

from fcutils.file_io.utils import check_create_folder, get_file_name
from fcutils.plotting.utils import clean_axes, save_figure
from fcutils.plotting.colors import *

from movie_visualizer import compare_videos
from utils import start_server, load_params, add_to_params_dict

bpl.output_notebook()

c, dview, n_processes = start_server()



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

INFO:caiman.cluster:Stopping cluster...
2020-04-15 14:59:05 PM - INFO - MainProcess cluster.py:301 - Stopping cluster...
INFO:caiman.cluster:No cluster to stop...
2020-04-15 14:59:06 PM - INFO - MainProcess cluster.py:344 - No cluster to stop...
INFO:caiman.cluster:stop_cluster(): done
2020-04-15 14:59:06 PM - INFO - MainProcess cluster.py:358 - stop_cluster(): done
A cluster is already running


## Get file paths

In [2]:
# Get files to process
fld = "D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05" # main data folder

fnames    = [os.path.join(fld, '19JUN05_BF164p1_v1_ds126_crop_ffcSub.tiff')]  # video to processprocessed
base_name = pathlib.Path(fnames[0]).stem # used to save mmepped data

output_fld = os.path.join(fld, "MC_output") # plots and other stuff will be saved here
check_create_folder(output_fld)



## Set up MC params

Each session's parameters are saved in fld > 01_PARAMS > params.yml

In [3]:
# dataset dependent parameters
frate = 10.                       # movie frame rate
decay_time = 2.              # length of a typical transient in seconds


# Load recording specific params and fill them up
prms = load_params(fld)['motion_correction']
prms = add_to_params_dict(prms, fnames=fnames, fr=frate, decay_time=decay_time)


opts = params.CNMFParams(params_dict=prms)


# Setup logging
fancylog.start_logging(os.path.join(fld, "02_LOGS"), package, file_log_level="INFO", variables=[opts], verbose=True, filename='motion_correction_logs')
logging.info("Starting motion correction")
logging.info(f"Output folder: {output_fld}")

INFO:root:Starting logging
2020-04-15 14:47:06 PM - INFO - MainProcess fancylog.py:271 - Starting logging
INFO:root:Multiprocessing-logging module not found, not logging multiple processes.
2020-04-15 14:47:06 PM - INFO - MainProcess fancylog.py:273 - Multiprocessing-logging module not found, not logging multiple processes.
INFO:root:Starting motion correction
2020-04-15 14:47:06 PM - INFO - MainProcess <ipython-input-3-e5e76afa20df>:16 - Starting motion correction
INFO:root:Output folder: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\MC_output
2020-04-15 14:47:06 PM - INFO - MainProcess <ipython-input-3-e5e76afa20df>:17 - Output folder: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\MC_output


# Motion Correction
The background signal in micro-endoscopic data is very strong and makes the motion correction challenging. 
As a first step the algorithm performs a high pass spatial filtering with a Gaussian kernel to remove the bulk of the background and enhance spatial landmarks. 
The size of the kernel is given from the parameter `gSig_filt`. If this is left to the default value of `None` then no spatial filtering is performed (default option, used in 2p data).
After spatial filtering, the NoRMCorre algorithm is used to determine the motion in each frame. The inferred motion is then applied to the *original* data so no information is lost.



## Rigid motion correction

In [4]:
mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))

In [5]:
# correct for rigid motion correction and save the file (in memory mapped form)
_ = mc.motion_correct(save_movie=True)


2020-04-15 14:47:08 PM - WARNING - MainProcess movies.py:371 - Movie average is negative. Removing 1st percentile.
2020-04-15 14:47:08 PM - WARNING - MainProcess movies.py:389 - Movie average is negative. Removing 1st percentile.
2020-04-15 14:47:08 PM - WARNING - MainProcess movies.py:371 - Movie average is negative. Removing 1st percentile.
2020-04-15 14:47:08 PM - WARNING - MainProcess movies.py:389 - Movie average is negative. Removing 1st percentile.
2020-04-15 14:47:08 PM - WARNING - MainProcess movies.py:371 - Movie average is negative. Removing 1st percentile.
2020-04-15 14:47:08 PM - WARNING - MainProcess movies.py:389 - Movie average is negative. Removing 1st percentile.
INFO:root:Saving file as D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub._rig__d1_109_d2_92_d3_1_order_F_frames_22662_.mmap
2020-04-15 14:47:08 PM - INFO - MainProcess motion_correction.py:3017 - Saving file as D:\Dropbox (UCL - SWC)\Project_vgatPAG\a

In [6]:
# Now save in C order for CNMF-E 
bord_px = 0
bord_px = 0 if prms['border_nan'] is 'copy' else bord_px
fname_new = cm.mmapping.save_memmap(mc.mmap_file, base_name=base_name+"_rig_", order='C', border_to_0=bord_px)


logging.info(f"Rigid motion corrected video was saved at:\n {fname_new}")

INFO:root:Rigid motion corrected video was saved at:
 D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_rig__d1_109_d2_92_d3_1_order_C_frames_22662_.mmap
2020-04-15 14:48:54 PM - INFO - MainProcess <ipython-input-6-2ea9de795875>:7 - Rigid motion corrected video was saved at:
 D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_rig__d1_109_d2_92_d3_1_order_C_frames_22662_.mmap


### Piecewise rigid motion correction

In [7]:
# motion correct piecewise rigid
mc.pw_rigid = True  # turn the flag to True for pw-rigid motion correction
mc.template = mc.mmap_file  # use the results of the rigid motion corrction to save in computation
_ = mc.motion_correct(save_movie=True, template=mc.total_template_rig)



INFO:root:Saving file as D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub._els__d1_109_d2_92_d3_1_order_F_frames_22662_.mmap
2020-04-15 14:48:55 PM - INFO - MainProcess motion_correction.py:3017 - Saving file as D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub._els__d1_109_d2_92_d3_1_order_F_frames_22662_.mmap
INFO:root:** Starting parallel motion correction **
2020-04-15 14:48:55 PM - INFO - MainProcess motion_correction.py:3030 - ** Starting parallel motion correction **
INFO:root:** Finished parallel motion correction **
2020-04-15 14:50:25 PM - INFO - MainProcess motion_correction.py:3038 - ** Finished parallel motion correction **


In [8]:
# Now save in C order for CNMF-E 
bord_px = 0 if prms['border_nan'] is 'copy' else bord_px
fname_new_els = cm.mmapping.save_memmap(mc.mmap_file, base_name=base_name+"_els_", order='C', border_to_0=bord_px)

logging.info(f"Piecewise motion corrected video was saved at:\n {fname_new_els}")

INFO:root:Piecewise motion corrected video was saved at:
 D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_els__d1_109_d2_92_d3_1_order_C_frames_22662_.mmap
2020-04-15 14:50:35 PM - INFO - MainProcess <ipython-input-8-9f6012b0e865>:5 - Piecewise motion corrected video was saved at:
 D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_els__d1_109_d2_92_d3_1_order_C_frames_22662_.mmap


# MC quality evaluations

Visualize the results in plots and video and compute metrics to quantify the quality of the motion correction.


In [9]:
MC_TYPE = "els" # els or rigid mc
if MC_TYPE == "els":
    eval_video = fname_new_els 
else:
    eval_video = fname_new

In [10]:
# load motion corrected movie
m_rig = cm.load(eval_video)

# visualize templates
plt.figure(figsize = (20,10))
_ = plt.imshow(mc.total_template_rig, cmap = 'gray')


2020-04-15 14:52:40 PM - WARNING - MainProcess _qt.py:187 - Attribute Qt::AA_EnableHighDpiScaling must be set before QCoreApplication is created.


## Look at correlation and PNR in motion corrected movie

You can use this to find the correct values for `min_corr` and `min_pnr` values for cnmf fitting.

In [11]:
Yr, dims, T = cm.load_memmap(fname_new_els)
images = Yr.T.reshape((T,) + dims, order='F')

# compute some summary images (correlation and peak to noisae)
cn_filter, pnr = cm.summary_images.correlation_pnr(images, gSig=opts.init['gSig'][0], swap_dim=False) 

# inspect the summary images and set the parameters
inspect_correlation_pnr(cn_filter, pnr)

2020-04-15 14:53:20 PM - WARNING - MainProcess _qt.py:187 - Attribute Qt::AA_EnableHighDpiScaling must be set before QCoreApplication is created.


### Plot background
Crate a figure with background and other images from MC video and save to file

In [12]:
f, axarr = plt.subplots(ncols=4, figsize=(20, 8))
imgs = [mc.total_template_rig, np.median(images, axis=0), cn_filter, pnr]
ttls = ["templates", "median bg", "cn_filter", "pnr"]
for ax, img, ttl in zip(axarr, imgs, ttls):
    ax.imshow(img, cmap="viridis")
    ax.set(title=ttl)
clean_axes(f)
save_figure(f, os.path.join(output_fld, MC_TYPE+"_bgs"))

2020-04-15 14:53:35 PM - WARNING - MainProcess _qt.py:187 - Attribute Qt::AA_EnableHighDpiScaling must be set before QCoreApplication is created.
 saved figure at: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\MC_output\els_bgs


## Inspect videos
In addition to these plots, it's worth using `compare_videos` in `movie_visualizer.py` to compare raw, rigid mc and pw mc videos:

```compare_videos(raw = fnames[0], rigid=fname_new, piecewise=fname_new_els)``` [better to do it outside of this ntotebook as it might block the kernel, you can find the path to the files in the jupyter notebook.]


In [13]:
logging.info(f"\n\nRaw video {fnames[0]}\n\nRigid mc: {fname_new}\n\nPiecewise mc: {fname_new_els}\n\n")

INFO:root:

Raw video D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub.tiff

Rigid mc: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_rig__d1_109_d2_92_d3_1_order_C_frames_22662_.mmap

Piecewise mc: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_els__d1_109_d2_92_d3_1_order_C_frames_22662_.mmap


2020-04-15 14:53:50 PM - INFO - MainProcess <ipython-input-13-3e49af22db9f>:1 - 

Raw video D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub.tiff

Rigid mc: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_rig__d1_109_d2_92_d3_1_order_C_frames_22662_.mmap

Piecewise mc: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_els__d1_109_d2_92_d3_1_order_C

## Compute quality metrics

In [34]:
#% compute metrics for the results (TAKES TIME!!)
final_size = np.subtract(mc.total_template_els.shape, 2 * bord_px) # remove pixels in the boundaries
winsize = 100
swap_dim = False
resize_fact_flow = .2    # downsample for computing ROF

tmpl_rig, correlations_orig, flows_orig, norms_orig, crispness_orig = cm.motion_correction.compute_metrics_motion_correction(
    fnames[0], final_size[0], final_size[1], swap_dim, winsize=winsize, play_flow=False, resize_fact_flow=resize_fact_flow)

tmpl_rig, correlations_rig, flows_rig, norms_rig, crispness_rig = cm.motion_correction.compute_metrics_motion_correction(
    mc.fname_tot_rig[0], final_size[0], final_size[1],
    swap_dim, winsize=winsize, play_flow=False, resize_fact_flow=resize_fact_flow)

tmpl_els, correlations_els, flows_els, norms_els, crispness_els = cm.motion_correction.compute_metrics_motion_correction(
    mc.fname_tot_els[0], final_size[0], final_size[1],
    swap_dim, winsize=winsize, play_flow=False, resize_fact_flow=resize_fact_flow)        

INFO:root:[0, None, 0, None]
2020-04-15 15:13:05 PM - INFO - MainProcess motion_correction.py:2526 - [0, None, 0, None]
INFO:root:Compute optical flow .. 
2020-04-15 15:13:24 PM - INFO - MainProcess motion_correction.py:2558 - Compute optical flow .. 
INFO:root:[0, None, 0, None]
2020-04-15 15:14:07 PM - INFO - MainProcess motion_correction.py:2526 - [0, None, 0, None]
INFO:root:Compute optical flow .. 
2020-04-15 15:14:27 PM - INFO - MainProcess motion_correction.py:2558 - Compute optical flow .. 
INFO:root:[0, None, 0, None]
2020-04-15 15:15:10 PM - INFO - MainProcess motion_correction.py:2526 - [0, None, 0, None]
INFO:root:Compute optical flow .. 
2020-04-15 15:15:30 PM - INFO - MainProcess motion_correction.py:2558 - Compute optical flow .. 


### Correlation
Look at the correlation between each frame and a reference frame (e.g. avg frame across entire video). If the motion correction worked, the correlation of the motion corrected frames should be higher than that of the raw frame (if stuff moves frames wont't be correlated). This metric is in part influenced by neural activity but not too much. 

In [27]:
# Plot correlation with mean frame
f = plt.figure(figsize = (20,10))

# Plot correlation to mean frame for all frames
ax = plt.subplot(211)
ax.plot(correlations_orig, color=goldenrod, label="original", lw=3, alpha=1)
ax.plot(correlations_rig, color=darkseagreen, label="rigid", lw=2, alpha=.8)
ax.plot(correlations_els, color=salmon, label="piecewise", lw=2, alpha=.6)
ax.set(title="Frame by frame correlation to mean frame", xlabel="frame", ylabel="correlation")
ax.legend()

# Plot original vs rigid correlation
ax = plt.subplot(223)
ax.scatter(correlations_orig, correlations_rig, color=darkseagreen, alpha=.3)
ax.plot([0, 1], [0, 1], '--', lw=2, alpha=.8, color=[.4, .4, .4])
ax.set(title="Frame by frame correlation to mean frame", xlabel="original", ylabel="rigid",
            xlim=[.3, .7], ylim=[.3, .7])
ax.axis('square')

# Plot rogod vs piecewise
ax = plt.subplot(224)
ax.scatter(correlations_orig, correlations_els, color=salmon, alpha=.3)
ax.plot([0, 1], [0, 1], '--', lw=2, alpha=.8, color=[.4, .4, .4])
ax.set(title="Frame by frame correlation to mean frame", xlabel="original", ylabel="poecewose",
            xlim=[.3, .7], ylim=[.3, .7])
_ = ax.axis('square')

# save
clean_axes(f)
save_figure(f, os.path.join(output_fld, "frames_correlation_to_reference"))

2020-04-15 15:09:29 PM - WARNING - MainProcess _qt.py:187 - Attribute Qt::AA_EnableHighDpiScaling must be set before QCoreApplication is created.
 saved figure at: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\MC_output\frames_correlation_to_reference


### Crispness
Another metric is cripsness. If the motion correction worked. The average frame should be crisper (less blurry). 

In [32]:
# print crispness values
msg = f"Crispness:\n  original: {int(crispness_orig)}\n  rigid:  {int(crispness_rig)}\n  piecewise: {int(crispness_els)}"
logging.info(msg)




INFO:root:Crispness:
  original: 169
  rigid:  161
  piecewise: 166
2020-04-15 15:12:28 PM - INFO - MainProcess <ipython-input-32-edbb2de397c1>:3 - Crispness:
  original: 169
  rigid:  161
  piecewise: 166


### Optic flow
Finally the last metric checks the optic flow across the video. If the video has been correctly motion corrected, the residual optic flow should be minimal. 

In [87]:
# plot the results of Residual Optical Flow
ttles = ["raw", "rigid", "piecewise"]
files = [fnames[0], mc.fname_tot_rig[0], mc.fname_tot_els[0]]
metric_files = [os.path.join(fld, get_file_name(f)+"._metrics.npz") for f in files]


f, axarr = plt.subplots(figsize = (20,10), ncols=3, nrows=3)

for i, (fl, ttl) in enumerate(zip(metric_files, ttles)):
    if not os.path.isfile(fl):
        raise ValueError
    else:
        ld = np.load(fl)
        
    if fl.endswith("mmap"):
        mean_img = np.mean(cm.load(fl[:-12] + 'mmap'), 0)[12:-12, 12:-12]
    else:
        mean_img = np.mean(cm.load(fnames[0]), 0)[12:-12, 12:-12]


    lq, hq = np.nanpercentile(mean_img, [.5, 99.5])

    axarr[i, 0].imshow(mean_img, vmin=lq, vmax=hq)
    axarr[i, 0].set(title="Mean image " + ttl)

    axarr[i, 1].imshow(ld['img_corr'], vmin=0.99, vmax=1)
    axarr[i, 1].set(title='Corr image ' + ttl)

    flows = ld['flows']
    img = axarr[i, 2].imshow(np.mean(np.sqrt(flows[:, :, :, 0]**2 + flows[:, :, :, 1]**2), 0), vmin=0.1, vmax=0.3)

    axarr[i, 2].set(title="mean optical flow " + ttl)

# save
f.tight_layout()
clean_axes(f)
save_figure(f, os.path.join(output_fld, "optic_flow_summary"))

2020-04-15 15:39:59 PM - WARNING - MainProcess _qt.py:187 - Attribute Qt::AA_EnableHighDpiScaling must be set before QCoreApplication is created.
 saved figure at: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\MC_output\optic_flow_summary


In [ ]:
# TODO add an option to just save the optic stuff as raw image because I will want to overlay the contours on top, or maybe just make a utility to load and generate image easily

# Stop cluster

In [ ]:
cm.stop_server(dview=dview)